# Standalone libCEED examples

This is a tutorial for [libCEED](https://github.com/CEED/libCEED/), the low-level API library for efficient high-order discretization methods developed by the co-design [Center for Efficient Exascale Discretizations](https://ceed.exascaleproject.org/) (CEED) of the [Exascale Computing Project](https://www.exascaleproject.org/) (ECP).

While libCEED's focus is on high-order finite/spectral element method implementations, the approach is mostly algebraic and thus applicable to other discretizations in factored form, as explained in the [user manual](https://libceed.readthedocs.io/) and [Doxygen documentation](https://codedocs.xyz/CEED/libCEED/).

## Common notation

For most of our examples, the spatial discretization
uses high-order finite elements/spectral elements, namely, the high-order Lagrange
polynomials defined over $P$ non-uniformly spaced nodes, the
Gauss-Legendre-Lobatto (GLL) points, and quadrature points $\{q_i\}_{i=1}^Q$, with
corresponding weights $\{w_i\}_{i=1}^Q$ (typically the ones given by Gauss
or Gauss-Lobatto quadratures, that are built in the library).

We discretize the domain, $\Omega \subset \mathbb{R}^d$ (with $d=1,2,3$,
typically) by letting $\Omega = \bigcup_{e=1}^{N_e}\Omega_e$, with $N_e$
disjoint elements. For most examples we use unstructured meshes for which the elements
are hexahedra (although this is not a requirement in libCEED).

The physical coordinates are denoted by $\mathbf{x}=(x,y,z)\in\Omega_e$,
while the reference coordinates are represented as
$\boldsymbol{X}=(X,Y,Z) \equiv (X_1,X_2,X_3) \in\mathbf{I}=[-1,1]^3$
(for $d=3$).

### Ex1-Volume

#### Mathematical formulation

This example is located in the subdirectory `examples/ceed`. It illustrates a
simple usage of libCEED to compute the volume of a given body using a matrix-free
application of the mass operator. Arbitrary mesh and solution orders in 1D, 2D and 3D
are supported from the same code.

This example shows how to compute line/surface/volume integrals of a 1D, 2D, or 3D
domain $\Omega$ respectively, by applying the mass operator to a vector of
$\mathbf{1}$s. It computes:

$$
   I = \int_{\Omega} \mathbf{1} \, d V\, .
$$

We write here the vector $u(\mathbf{x})\equiv \mathbf{1}$ in the Galerkin approximation,
and find the volume of $\Omega$ as

$$
   \sum_e \int_{\Omega_e} v(x) \cdot \mathbf{1} \, dV
$$

with $v(x) \in \mathcal{V}_p = \{ v \in H^{1}(\Omega_e) \,|\, v \in P_p(\boldsymbol{I}), e=1,\ldots,N_e \}$,
the test functions.


#### Executing the example

Clone or download libCEED by running

In [ ]:
! git clone https://github.com/CEED/libCEED.git

Then move to the libCEED folder

In [ ]:
cd libCEED

And compile the library by running

In [ ]:
! make

Move to the examples folder 

In [ ]:
cd examples/

Then move to the standalone libCEED's examples folder

In [ ]:
cd ceed/

And compile the examples by running

In [ ]:
! make

Now run `ex1-volume` by running

In [1]:
! ./ex1-volume -d 3 -g

Selected options: [command line option] : <current value>
  Ceed specification [-c] : /cpu/self
  Mesh dimension     [-d] : 3
  Mesh order         [-m] : 4
  Solution order     [-o] : 4
  Num. 1D quadr. pts [-q] : 6
  Approx. # unknowns [-s] : 262144
  QFunction source   [-g] : gallery

Mesh size: nx = 16, ny = 16, nz = 16
Number of mesh nodes     : 274625
Number of solution nodes : 274625
Computing the quadrature data for the mass operator ... done.
Computing the mesh volume using the formula: vol = 1^T.M.1 ... done.
Exact mesh volume    :  2.3561944901923
Computed mesh volume :  2.3561944901921
Volume error         : -2.7444713168734e-13


This example shows how to compute line/surface/volume integrals of a 1D, 2D, or 3D domain Ω respectively, by applying the mass operator to a vector of 1s. The command line option `-d` specifies the dimensionality of the domain Ω. The option `-g` specifies that the mass operator is, in this case, selected from a gallery of available built-in operators in the library.

### Ex2-Surface

#### Mathematical formulation

This example is located in the subdirectory `examples/ceed`. It computes the
surface area of a given body using matrix-free application of a diffusion operator.
Arbitrary mesh and solution orders in 1D, 2D and 3D are supported from the same code.

Similarly to the Ex1-Volume example, it computes:

\begin{equation}\label{eq-ex2-surface}\tag{eq. 1}
   I = \int_{\partial \Omega} \mathbf{1} \, d S\, .
\end{equation}

but this time by solving a Laplace's equation for a harmonic function
$u(\mathbf{x})$. We write the Laplace's equation

$$
   \nabla \cdot \nabla u = 0, \textrm{ for  } \mathbf{x} \in \Omega.
$$

We can rewrite this via the bilinear form $a(\cdot,\cdot)$ and the linear form $\langle \cdot, \cdot \rangle$ as

$$
   a(u,v) = \langle v, f \rangle
$$

where $v$ is the test function, and for which $\langle v, f \rangle=0$ in this case. We
obtain

$$
   a(u,v)  = \int_\Omega v \nabla \cdot \nabla u \, d V =   \int_{\partial \Omega} v \nabla u \cdot \mathbf{n}\, d S - \int_\Omega \nabla v \cdot \nabla u \, d V  = 0\, ,
$$

where we have used integration by parts.

$a(u,v) = 0$ because we have chosen $u(\mathbf{x})$ to be harmonic, so we
can write

\begin{equation}\label{eq-laplace-by-parts}\tag{eq. 2}
   \int_{\partial \Omega} v \nabla u \cdot \mathbf{n}\, d S = \int_\Omega \nabla v \cdot \nabla u \, d V
\end{equation}

and use the `CeedOperator` for the Laplacian to compute the right-hand side of 
\ref{eq-laplace-by-parts}. This way, the left-hand side of 
\ref{eq-laplace-by-parts} (which gives \ref{eq-ex2-surface} because
we have chosen $u(\mathbf{x}) = (x + y + z)$ such that $\nabla u \cdot \mathbf{n} = 1$)
is readily found.

### Executing the example

Assuming the steps above, you should be in the `examples/ceed/` subdirectory and have already compiled the example.

Now run `ex2-surface` by running 

In [2]:
! ./ex2-surface -d 3 -g

Selected options: [command line option] : <current value>
  Ceed specification [-c] : /cpu/self
  Mesh dimension     [-d] : 3
  Mesh order         [-m] : 4
  Solution order     [-o] : 4
  Num. 1D quadr. pts [-q] : 6
  Approx. # unknowns [-s] : 262144
  QFunction source   [-g] : gallery

Mesh size: nx = 16, ny = 16, nz = 16
Number of mesh nodes     : 274625
Number of solution nodes : 274625
Computing the quadrature data for the diffusion operator ... done.
Computing the mesh surface area using the formula: sa = 1^T.|K.x| ... done.
Exact mesh surface area    :  6
Computed mesh surface area :  5.9773703490853
Surface area error         : -0.022629650914673


This example computes the surface area of a given body using matrix-free application of a Laplace's (diffusion) operator. The command line option `-d` specifies the dimensionality of the domain Ω. The option `-g` specifies that the Laplace's operator is, in this case, selected from a gallery of available built-in operators in the library.